In [1]:
# ===============================
# Demand Agent: Data Prep + XGBoost Forecast + Save Model
# ===============================

import pandas as pd
import glob
import os
import pickle
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

# ----------------------------
# 1️⃣ Read and concatenate consumption data
# ----------------------------
consumption_files = glob.glob("Consumption Data/consumption_*.csv")  # replace with your path
consumption_list = []

for file in consumption_files:
    df = pd.read_csv(file)
    df['Date'] = pd.to_datetime(df['Date'])
    consumption_list.append(df)

consumption_data = pd.concat(consumption_list, ignore_index=True)
consumption_data = consumption_data[['Date', 'Inventory_ID', 'Quantity_Consumed']]

# ----------------------------
# 2️⃣ Read and concatenate inventory data
# ----------------------------
# ----------------------------
# 2️⃣ Read and concatenate inventory data
# ----------------------------
inventory_files = glob.glob("Inventory Data/inventory_*.csv")  # replace with your path
inventory_list = []

for file in inventory_files:
    df = pd.read_csv(file)
    df['Date'] = pd.to_datetime(df['Date'])
    inventory_list.append(df)

inventory_data = pd.concat(inventory_list, ignore_index=True)

# Keep Item_Name here
inventory_data = inventory_data[['Date', 'Inventory_ID', 'Item_Name', 'Opening_Stock', 'Closing_Stock', 'Quantity_Restocked', 'Lead_Time_Days']]




In [2]:
# ----------------------------
# 3️⃣ Read master inventory list
# ----------------------------
master_file = "master_inventory_mapping.csv"  # replace with your path
master_data = pd.read_csv(master_file)
master_data.rename(columns={'inventory_id':'Inventory_ID'}, inplace=True)
master_data = master_data[['Inventory_ID', 'lead_time_days', 'min_stock_limit', 'max_capacity']]

# ----------------------------
# 4️⃣ Merge datasets
# ----------------------------
df = pd.merge(consumption_data, inventory_data, on=['Date','Inventory_ID'], how='left')
df = pd.merge(df, master_data, on='Inventory_ID', how='left')


# ----------------------------
# Save full dataset for MCP
# ----------------------------
os.makedirs("models", exist_ok=True)  # optional folder
mcp_data_path = "models/demand_forecast_base.csv"
df.to_csv(mcp_data_path, index=False)
print(f"✅ Full dataset saved for MCP at {mcp_data_path}")



✅ Full dataset saved for MCP at models/demand_forecast_base.csv


In [3]:
# ----------------------------
# 5️⃣ Feature engineering
# ----------------------------
df = df.sort_values(['Inventory_ID','Date']).reset_index(drop=True)

# Lag features: last 7 days consumption
for lag in range(1, 8):
    df[f'lag_{lag}'] = df.groupby('Inventory_ID')['Quantity_Consumed'].shift(lag)

# Date features
df['day_of_week'] = df['Date'].dt.dayofweek
df['month'] = df['Date'].dt.month

# Drop rows with NaN (first few rows will have NaNs due to lag)
df_model = df.dropna().reset_index(drop=True)

# Features & target
feature_cols = ['Opening_Stock','Closing_Stock','Quantity_Restocked','Lead_Time_Days',
                'lead_time_days','min_stock_limit','max_capacity','day_of_week','month'] + [f'lag_{i}' for i in range(1,8)]
target_col = 'Quantity_Consumed'

X = df_model[feature_cols]
y = df_model[target_col]



In [4]:
# ----------------------------
# 6️⃣ Train-test split
# ----------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# ----------------------------
# 7️⃣ Train XGBoost model
# ----------------------------
xgb_model = XGBRegressor(n_estimators=300, learning_rate=0.05, max_depth=6, random_state=42)
xgb_model.fit(X_train, y_train)

# Evaluate (optional)
print("Train R^2:", xgb_model.score(X_train, y_train))
print("Test R^2:", xgb_model.score(X_test, y_test))

# ----------------------------
# 8️⃣ Save trained model
# ----------------------------
# Save trained model as JSON (recommended)
model_path = "models/demand_agent_xgb.json"
xgb_model.save_model(model_path)
print(f"✅ Model saved to {model_path}")

# ----------------------------
# 9️⃣ Forecast next 30 days per item
# ----------------------------
forecast_list = []
for item_id in df['Inventory_ID'].unique():
    item_df = df[df['Inventory_ID']==item_id].sort_values('Date').copy()
    
    # Prepare last available row to start forecasting
    last_row = item_df.iloc[-1].copy()
    preds = []

    for day in range(30):
        # Build feature vector
        feat = {
            'Opening_Stock': last_row['Closing_Stock'],
            'Closing_Stock': last_row['Closing_Stock'],
            'Quantity_Restocked': 0,  # assume unknown
            'Lead_Time_Days': last_row['Lead_Time_Days'],
            'lead_time_days': last_row['lead_time_days'],
            'min_stock_limit': last_row['min_stock_limit'],
            'max_capacity': last_row['max_capacity'],
            'day_of_week': (last_row['Date'].dayofweek + 1) % 7,
            'month': ((last_row['Date'].month + (day//30)) % 12) +1
        }
        # Add lag features
        for lag in range(1,8):
            feat[f'lag_{lag}'] = last_row['Quantity_Consumed'] if lag==1 else last_row[f'lag_{lag-1}']

        # Predict
        X_pred = pd.DataFrame([feat])
        y_pred = xgb_model.predict(X_pred)[0]
        y_pred = max(0, y_pred)  # no negative consumption

        # Save prediction
        forecast_list.append({'Date': last_row['Date'] + pd.Timedelta(days=day+1),
                              'Inventory_ID': item_id,
                              'Predicted_Consumption': y_pred})

        # Update last_row for next iteration
        for lag in range(7,1,-1):
            last_row[f'lag_{lag}'] = last_row[f'lag_{lag-1}']
        last_row['lag_1'] = y_pred
        last_row['Closing_Stock'] = last_row['Closing_Stock'] - y_pred  # assume no restock for simplicity

forecast_df = pd.DataFrame(forecast_list)
forecast_df.to_csv("demand_forecast_xgb.csv", index=False)
print("✅ Forecast saved to demand_forecast_xgb.csv")


Train R^2: 0.9010257124900818
Test R^2: 0.8621034026145935
✅ Model saved to models/demand_agent_xgb.json
✅ Forecast saved to demand_forecast_xgb.csv
